In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 31.6 MB/s 
     |████████████████████████████████| 182 kB 81.1 MB/s 
     |████████████████████████████████| 7.6 MB 68.5 MB/s 


In [4]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 19.9 MB/s 
     |████████████████████████████████| 213 kB 67.1 MB/s 
     |████████████████████████████████| 132 kB 64.7 MB/s 
     |████████████████████████████████| 127 kB 62.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
data = pd.read_csv("/content/drive/MyDrive/train_data.csv", on_bad_lines='skip')

In [6]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
#preprocessing
def preprocessing(data):
  import nltk
  from nltk.stem.porter import PorterStemmer
  import re
  from nltk.corpus import stopwords
  ps = PorterStemmer()
  sentences = []
  nltk.download("stopwords")

  for i in range(data.shape[0]): # data.shape[0]
    sentence = re.sub('[^a-zA-Z]',' ',data.iloc[i].values[0]) # noktalama işaretlerini silme.
    sentence = sentence.lower() # kelimelerin hepsini küçük harf 
    sentence = sentence.split() # keliemeleri listeye atma
    sentence = [ps.stem(kelime) for kelime in sentence if not kelime in set(stopwords.words("english"))] #anlamı olmayan(stopwords) kelimeleri listeden atma
    sentence = " ".join(sentence) # listede kalan kelimeleri birleştirip yeni cümleyi oluşturma.
    sentences.append(sentence)
  df = pd.DataFrame(data = sentences,columns = ["Haber"])
  df["Truth"] = data["Truth"]
  return df

In [9]:
preprocessing_data = preprocessing(data)
preprocessing_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Haber,Truth
0,undercov video prove trump right voter fraud n...,0
1,club face fa cup first round,1
2,world cup injuri watch doubt,1
3,statu world cup winner charlton unveil,1
4,polic apolog pursu sex assault report,1
...,...,...
42198,turkey close border iraq evacu embassi,1
42199,storm arwen damag tree use help save salmon,1
42200,iraq war veteran take cowardli republican wood...,0
42201,deplor hillari got bust offend half basecal be...,0


In [10]:
def process_data(row):
    sentence = row[0]
    truth_value = row[1]
    sentence = " ".join(sentence.split())
    encodings = tokenizer(sentence,padding = "max_length",truncation =True,max_length = 128)
    encodings["label"] = truth_value
    encodings["text"] = sentence
    return encodings

In [11]:
processed_data = []
for i in range(0,preprocessing_data.shape[0]):
    processed_data.append(process_data(preprocessing_data.iloc[i]))

In [12]:
veri = pd.DataFrame(data = processed_data)
veri

,attention_mask,input_ids,label,text
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2104, 3597, 2615, 2678, 6011, 8398, 2157...",0,undercov video prove trump right voter fraud n...
1,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[101, 2252, 2227, 6904, 2452, 2034, 2461, 102,...",1,club face fa cup first round
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[101, 2088, 2452, 1999, 9103, 3089, 3422, 4797...",1,world cup injuri watch doubt
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[101, 28093, 2226, 2088, 2452, 3453, 17821, 48...",1,statu world cup winner charlton unveil
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[101, 14955, 2594, 9706, 12898, 2290, 16405, 2...",1,polic apolog pursu sex assault report
...,...,...,...,...
42198,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[101, 4977, 2485, 3675, 5712, 9345, 10841, 786...",1,turkey close border iraq evacu embassi
42199,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[101, 4040, 12098, 12449, 5477, 8490, 3392, 22...",1,storm arwen damag tree use help save salmon
42200,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[101, 5712, 2162, 8003, 2202, 16592, 3669, 395...",0,iraq war veteran take cowardli republican wood...
42201,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2139, 24759, 2953, 2940, 8486, 2288, 139...",0,deplor hillari got bust offend half basecal be...


In [13]:
from sklearn.model_selection import train_test_split
train_df,valid_df = train_test_split(veri,test_size = 0.2)

In [14]:
import pyarrow as pa
from datasets import Dataset
train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name) #,num_labels=2

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [16]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/result",evaluation_strategy="epoch")
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer = tokenizer,
)

In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 33762
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12663
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, u

Epoch,Training Loss,Validation Loss
1,0.097300,0.064256
2,0.038700,0.061104
3,0.008500,0.099738


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/result/checkpoint-500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/result/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/result/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/result/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/result/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english

TrainOutput(global_step=12663, training_loss=0.05281463865795887, metrics={'train_runtime': 1484.6811, 'train_samples_per_second': 68.221, 'train_steps_per_second': 8.529, 'total_flos': 3354273235067904.0, 'train_loss': 0.05281463865795887, 'epoch': 3.0})

In [18]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8441
  Batch size = 8


{'eval_loss': 0.09973820298910141,
 'eval_runtime': 30.3069,
 'eval_samples_per_second': 278.517,
 'eval_steps_per_second': 34.844,
 'epoch': 3.0}

In [19]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/model")

Configuration saved in /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/model/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/distilbert-base-uncased-finetuned-sst-2-english/model/pytorch_model.bin
